In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ExtracaoCSV") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.2,com.amazonaws:aws-java-sdk-bundle:1.11.1026") \
    .config("fs.s3a.endpoint", "http://172.18.0.4:9000") \
    .config("fs.s3a.access.key", "minioaccesskey") \
    .config("fs.s3a.secret.key", "miniosecretkey") \
    .config("fs.s3a.path.style.access", "true") \
    .config("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("fs.s3a.connection.ssl.enabled", "false") \
    .getOrCreate()

In [2]:
# Informações de conexão Minio
minio_host = "http://localhost:9000"
minio_access_key = "minioaccesskey"
minio_secret_key = "miniosecretkey"
minio_bucket_bronze = "bronze"

hadoop_conf = spark.sparkContext._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3a.endpoint", minio_host)
hadoop_conf.set("fs.s3a.access.key", minio_access_key)
hadoop_conf.set("fs.s3a.secret.key", minio_secret_key)
hadoop_conf.set("fs.s3a.path.style.access", "true")
hadoop_conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
hadoop_conf.set("fs.s3a.connection.ssl.enabled", "false")

In [3]:
# Caminho para a pasta de dados CSV (relativo a /notebooks/)
csv_path = "dados/"

# Ler matricula_aulas.csv
file_matricula_aulas = csv_path + "matricula_aulas.csv"
df_matricula_aulas = spark.read.csv(file_matricula_aulas, header=True, inferSchema=True)
print("Schema matricula_aulas:")
df_matricula_aulas.printSchema()
df_matricula_aulas.show(5)

# Ler matricula_disciplinas.csv
file_matricula_disciplinas = csv_path + "matricula_disciplinas.csv"
df_matricula_disciplinas = spark.read.csv(file_matricula_disciplinas, header=True, inferSchema=True)
print("\nSchema matricula_disciplinas:")
df_matricula_disciplinas.printSchema()
df_matricula_disciplinas.show(5)

# Ler matricula_notas.csv
file_matricula_notas = csv_path + "matricula_notas.csv"
df_matricula_notas = spark.read.csv(file_matricula_notas, header=True, inferSchema=True)
print("\nSchema matricula_notas:")
df_matricula_notas.printSchema()
df_matricula_notas.show(5)

# Ler turma_avaliacoes.csv
file_turma_avaliacoes = csv_path + "turma_avaliacoes.csv"
df_turma_avaliacoes = spark.read.csv(file_turma_avaliacoes, header=True, inferSchema=True)
print("\nSchema turma_avaliacoes:")
df_turma_avaliacoes.printSchema()
df_turma_avaliacoes.show(5)

Schema matricula_aulas:
root
 |-- id_matricula: integer (nullable = true)
 |-- id_disciplina: integer (nullable = true)
 |-- data_aula: date (nullable = true)
 |-- presente: boolean (nullable = true)

+------------+-------------+----------+--------+
|id_matricula|id_disciplina| data_aula|presente|
+------------+-------------+----------+--------+
|          25|          501|2020-06-08|    true|
|          25|          501|2020-06-15|   false|
|          25|          501|2020-06-22|    true|
|          25|          501|2020-06-29|    true|
|          25|          501|2020-07-06|    true|
+------------+-------------+----------+--------+
only showing top 5 rows


Schema matricula_disciplinas:
root
 |-- id_matricula: integer (nullable = true)
 |-- id_disciplina: integer (nullable = true)

+------------+-------------+
|id_matricula|id_disciplina|
+------------+-------------+
|          25|          501|
|          25|          502|
|          25|          503|
|          25|          504|
| 

In [4]:
# Escrever df_matricula_aulas no Minio (Bronze)
path_matricula_aulas_bronze = f"s3a://{minio_bucket_bronze}/csv/matricula_aulas"
df_matricula_aulas.write.mode("overwrite").parquet(path_matricula_aulas_bronze)
print(f"Dados de matricula_aulas escritos em: {path_matricula_aulas_bronze}")

# Escrever df_matricula_disciplinas no Minio (Bronze)
path_matricula_disciplinas_bronze = f"s3a://{minio_bucket_bronze}/csv/matricula_disciplinas"
df_matricula_disciplinas.write.mode("overwrite").parquet(path_matricula_disciplinas_bronze)
print(f"Dados de matricula_disciplinas escritos em: {path_matricula_disciplinas_bronze}")

# Escrever df_matricula_notas no Minio (Bronze)
path_matricula_notas_bronze = f"s3a://{minio_bucket_bronze}/csv/matricula_notas"
df_matricula_notas.write.mode("overwrite").parquet(path_matricula_notas_bronze)
print(f"Dados de matricula_notas escritos em: {path_matricula_notas_bronze}")

# Escrever df_turma_avaliacoes no Minio (Bronze)
path_turma_avaliacoes_bronze = f"s3a://{minio_bucket_bronze}/csv/turma_avaliacoes"
df_turma_avaliacoes.write.mode("overwrite").parquet(path_turma_avaliacoes_bronze)
print(f"Dados de turma_avaliacoes escritos em: {path_turma_avaliacoes_bronze}")

# Parar a sessão Spark
spark.stop()

Dados de matricula_aulas escritos em: s3a://bronze/csv/matricula_aulas
Dados de matricula_disciplinas escritos em: s3a://bronze/csv/matricula_disciplinas
Dados de matricula_notas escritos em: s3a://bronze/csv/matricula_notas
Dados de turma_avaliacoes escritos em: s3a://bronze/csv/turma_avaliacoes
